### Practica Mongo airlines

La práctica consiste en lo siguiente:
Descargar el archivo ` wget 148.204.66.247/diplomado/airlines.json`
Generar un script en python que agrupe los datos por aeropuerto, año y mes y obtener todos los promedios del campo "Statistics", y el campo carrier tendrá una lista de documentos de todos los carrier que están involucrados con el promedio obtenido (todos lo carrier que se incluyen dentro de esta agrupación). El resultado se guardará en una base de datos de mongo llamada "aeropuertos" y en una colección con nombre "data". 

La agrupación y/o lectura del json la pueden realizar con python o cargar los datos previamente en Mongo y recuperarlos a través de una consulta a Mongo.

Tip:
Lo más sencillo es cargar los datos previamente a Mongo y mediante un aggregate realizar la agrupación, pueden revisar los siguientes operadores en Mongo (\\$aggregate, \\$group, \\$Avg, \\$push, \\$project).

Comenzamos creando la conexion a la base de datos.

In [1]:
import pymongo

micliente = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = micliente["practica"]
mycol = mydb["data"]


query = { "airport.code" : "ATL" , "time.label":"2003/6"}


for x in mycol.find(query,{"_id":0}):
    for y in x.keys():
        print(y,":" ,x[y])
        #print("\n")
    print("\n")

"""
for x in mycol.find(query):
    print(x)
    print("\n")
"""

airport : {'code': 'ATL', 'name': 'Atlanta, GA: Hartsfield-Jackson Atlanta International'}
statistics : {'flights': {'cancelled': 5, 'on time': 561, 'total': 752, 'delayed': 186, 'diverted': 0}, '# of delays': {'late aircraft': 18, 'weather': 28, 'security': 2, 'national aviation system': 105, 'carrier': 34}, 'minutes delayed': {'late aircraft': 1269, 'weather': 1722, 'carrier': 1367, 'security': 139, 'total': 8314, 'national aviation system': 3817}}
time : {'label': '2003/6', 'year': 2003, 'month': 6}
carrier : {'code': 'AA', 'name': 'American Airlines Inc.'}


airport : {'code': 'ATL', 'name': 'Atlanta, GA: Hartsfield-Jackson Atlanta International'}
statistics : {'flights': {'cancelled': 0, 'on time': 74, 'total': 90, 'delayed': 16, 'diverted': 0}, '# of delays': {'late aircraft': 2, 'weather': 0, 'security': 1, 'national aviation system': 11, 'carrier': 3}, 'minutes delayed': {'late aircraft': 36, 'weather': 0, 'carrier': 103, 'security': 17, 'total': 382, 'national aviation system'

'\nfor x in mycol.find(query):\n    print(x)\n    print("\n")\n'

In [40]:
"""
arreglo=[]
for x in mycol.find(query,{"carrier.name":1,"_id":0}):
    for y in x.keys():
        arreglo.append(x[y]['name'])

print(arreglo)
"""

['American Airlines Inc.', 'JetBlue Airways', 'Continental Air Lines Inc.', 'Delta Air Lines Inc.', 'Atlantic Southeast Airlines', 'AirTran Airways Corporation', 'America West Airlines Inc.', 'Northwest Airlines Inc.', 'ExpressJet Airlines Inc.', 'United Air Lines Inc.', 'US Airways Inc.']


In [64]:
query2 = [{ "$match" : {"time.label" : '2003/6'} },
{"$group" : { "_id": {"aeropuerto":"$airport.code"}  ,
         "avgFlightsCancelled" : {"$avg": "$statistics.flights.cancelled"},
         "avgOntime" : {"$avg": "$statistics.flights.on time"},
         "avgTotal" : {"$avg": "$statistics.flights.total"},
         "avgDelayed" : {"$avg": "$statistics.flights.delayed"},
         "avgDiverted" : {"$avg": "$statistics.flights.diverted"},
         "avgLateAircraft" : {"$avg": "$statistics.# of delays.late aircraft"},
         "airlines" : {"$push": "$carrier.code"}   } }  ]

cursor= mycol.aggregate(query2)
for i in list(cursor):
    print(i,"\n")

{'_id': {'aeropuerto': 'SLC'}, 'avgFlightsCancelled': 3.4, 'avgOntime': 949.1, 'avgTotal': 1058.2, 'avgDelayed': 105.7, 'avgDiverted': 0.0, 'avgLateAircraft': 29.0, 'airlines': ['AA', 'B6', 'CO', 'DL', 'EV', 'HP', 'NW', 'OO', 'UA', 'WN']} 

{'_id': {'aeropuerto': 'SFO'}, 'avgFlightsCancelled': 7.545454545454546, 'avgOntime': 746.4545454545455, 'avgTotal': 925.7272727272727, 'avgDelayed': 170.9090909090909, 'avgDiverted': 0.8181818181818182, 'avgLateAircraft': 28.363636363636363, 'airlines': ['AA', 'AS', 'CO', 'DL', 'HP', 'MQ', 'NW', 'OO', 'TZ', 'UA', 'US']} 

{'_id': {'aeropuerto': 'SEA'}, 'avgFlightsCancelled': 5.25, 'avgOntime': 638.1666666666666, 'avgTotal': 770.5, 'avgDelayed': 126.33333333333333, 'avgDiverted': 0.75, 'avgLateAircraft': 46.0, 'airlines': ['AA', 'AS', 'B6', 'CO', 'DL', 'HP', 'OO', 'NW', 'TZ', 'UA', 'US', 'WN']} 

{'_id': {'aeropuerto': 'PHX'}, 'avgFlightsCancelled': 4.75, 'avgOntime': 1013.1666666666666, 'avgTotal': 1167.3333333333333, 'avgDelayed': 148.583333333333

In [11]:
query2 = [{"$group" : { "_id": {"aeropuerto":"$airport.code", "periodo":"$time.label"}  , #"AÑO":"$time.year","MES":"$time.month"
         "avgFlightsCancelled" : {"$avg": "$statistics.flights.cancelled"},
         "avgOntime" : {"$avg": "$statistics.flights.on time"},
         "avgTotal" : {"$avg": "$statistics.flights.total"},
         "avgDelayed" : {"$avg": "$statistics.flights.delayed"},
         "avgDiverted" : {"$avg": "$statistics.flights.diverted"},
         "avgLateAircraft" : {"$avg": "$statistics.# of delays.late aircraft"},
         "avgWeather": {"$avg":"$statistics.# of delays.weather"},
         "avgSecurity": {"$avg":"$statistics.# of delays.security"},
         "avgNationalAviationSystem": {"$avg":"$statistics.# of delays.national aviation system"},
         "avgNumDelaysCarrier": {"$avg":"$statistics.# of delays.carrier"},              
         "avgMinuesDelayedLateAircraft": {"$avg":"$statistics.minutes delayed.late aircraft"},
         "avgMinuesDelayedWeather": {"$avg":"$statistics.minutes delayed.weather"},
         "avgMinuesDelayedCarrier": {"$avg":"$statistics.minutes delayed.carrier"},
         "avgMinuesDelayedSecurity": {"$avg":"$statistics.minutes delayed.security"},
         "avgMinuesDelayedTotal": {"$avg":"$statistics.minutes delayed.total"},
         "avgMinuesDelayedNatAviaSys": {"$avg":"$statistics.minutes delayed.national aviation system"},              
         "airlinesCodes" : {"$push": "$carrier.code"},
         "airlinesNames" : {"$push": "$carrier.name"} } }  ]

cursor= mycol.aggregate(query2)
for i in list(cursor):
    print(i,"\n")

{'_id': {'aeropuerto': 'MDW', 'periodo': '2016/1'}, 'avgFlightsCancelled': 38.5, 'avgOntime': 1419.5, 'avgTotal': 1669.0, 'avgDelayed': 207.75, 'avgDiverted': 3.25, 'avgLateAircraft': 85.0, 'avgWeather': 11.75, 'avgSecurity': 0.75, 'avgNationalAviationSystem': 51.75, 'avgNumDelaysCarrier': 58.25, 'avgMinuesDelayedLateAircraft': 4175.75, 'avgMinuesDelayedWeather': 853.75, 'avgMinuesDelayedCarrier': 2635.25, 'avgMinuesDelayedSecurity': 15.5, 'avgMinuesDelayedTotal': 9258.5, 'avgMinuesDelayedNatAviaSys': 1578.25, 'airlinesCodes': ['DL', 'EV', 'OO', 'WN'], 'airlinesNames': ['Delta Air Lines Inc.', 'ExpressJet Airlines Inc.', 'SkyWest Airlines Inc.', 'Southwest Airlines Co.']} 

{'_id': {'aeropuerto': 'MSP', 'periodo': '2016/1'}, 'avgFlightsCancelled': 10.555555555555555, 'avgOntime': 883.0, 'avgTotal': 1065.0, 'avgDelayed': 170.66666666666666, 'avgDiverted': 0.7777777777777778, 'avgLateAircraft': 62.22222222222222, 'avgWeather': 6.555555555555555, 'avgSecurity': 0.2222222222222222, 'avgNat

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Creamos el diccionario que va a contener la esructura de la colección de la nueva base de datos.


In [31]:
query2 = [{"$group" : { "_id": {"aeropuerto":"$airport.code", "periodo":"$time.label"}  , #"AÑO":"$time.year","MES":"$time.month"
         "avgFlightsCancelled" : {"$avg": "$statistics.flights.cancelled"},
         "avgOntime" : {"$avg": "$statistics.flights.on time"},
         "avgTotal" : {"$avg": "$statistics.flights.total"},
         "avgDelayed" : {"$avg": "$statistics.flights.delayed"},
         "avgDiverted" : {"$avg": "$statistics.flights.diverted"},
         "avgLateAircraft" : {"$avg": "$statistics.# of delays.late aircraft"},
         "avgWeather": {"$avg":"$statistics.# of delays.weather"},
         "avgSecurity": {"$avg":"$statistics.# of delays.security"},
         "avgNationalAviationSystem": {"$avg":"$statistics.# of delays.national aviation system"},
         "avgNumDelaysCarrier": {"$avg":"$statistics.# of delays.carrier"},              
         "avgMinuesDelayedLateAircraft": {"$avg":"$statistics.minutes delayed.late aircraft"},
         "avgMinuesDelayedWeather": {"$avg":"$statistics.minutes delayed.weather"},
         "avgMinuesDelayedCarrier": {"$avg":"$statistics.minutes delayed.carrier"},
         "avgMinuesDelayedSecurity": {"$avg":"$statistics.minutes delayed.security"},
         "avgMinuesDelayedTotal": {"$avg":"$statistics.minutes delayed.total"},
         "avgMinuesDelayedNatAviaSys": {"$avg":"$statistics.minutes delayed.national aviation system"},              
         "airlinesCodes" : {"$push": "$carrier.code"},
         "airlinesNames" : {"$push": "$carrier.name"} } },
         
         {
             "$project": {
                 "_id":1,
                 "Statistics" : { 
                     "flights" : {
                         "cancelled" : "$avgFlightsCancelled",
                         "on time"   : "$avgOntime",
                         "total"     : "$avgTotal",
                         "delayed"   : "$avgDelayed",
                         "diverted"  : "$avgDiverted"
                                 },
                     "# of delays": {
                         "late aircraft" : "$avgLateAircraft",
                         "weather" : "$avgWeather",
                         "security": "$avgSecurity",
                         "national aviation system":"$avgNationalAviationSystem",
                         "carrier": "$avgNumDelaysCarrier"
                         },
                     "minutes delayed" :{
                         "late aircraft":"$avgMinuesDelayedLateAircraft",
                         "weather":"$avgMinuesDelayedWeather",
                         "carrier" : "$avgMinuesDelayedCarrier",
                         "security": "$avgMinuesDelayedSecurity",
                         "total" : "$avgMinuesDelayedTotal",
                         "national aviation":"$avgMinuesDelayedNatAviaSys"
                     }
                 },
               "Carrier" : { "code": "$airlinesCodes", 
                             "name":"$airlinesNames"
                           }                    
               
             }
         }]

cursor= mycol.aggregate(query2)
cur=list(cursor)
for i in cur:
    print(i,"\n")


{'_id': {'aeropuerto': 'MDW', 'periodo': '2016/1'}, 'Statistics': {'flights': {'cancelled': 38.5, 'on time': 1419.5, 'total': 1669.0, 'delayed': 207.75, 'diverted': 3.25}, '# of delays': {'late aircraft': 85.0, 'weather': 11.75, 'security': 0.75, 'national aviation system': 51.75, 'carrier': 58.25}, 'minutes delayed': {'late aircraft': 4175.75, 'weather': 853.75, 'carrier': 2635.25, 'security': 15.5, 'total': 9258.5, 'national aviation': 1578.25}}, 'Carrier': {'code': ['DL', 'EV', 'OO', 'WN'], 'name': ['Delta Air Lines Inc.', 'ExpressJet Airlines Inc.', 'SkyWest Airlines Inc.', 'Southwest Airlines Co.']}} 

{'_id': {'aeropuerto': 'MSP', 'periodo': '2016/1'}, 'Statistics': {'flights': {'cancelled': 10.555555555555555, 'on time': 883.0, 'total': 1065.0, 'delayed': 170.66666666666666, 'diverted': 0.7777777777777778}, '# of delays': {'late aircraft': 62.22222222222222, 'weather': 6.555555555555555, 'security': 0.2222222222222222, 'national aviation system': 53.44444444444444, 'carrier': 48

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Creacion de BD 

Insertamos los resultados en la nueva base de datos...


In [34]:
mydb2 = micliente['aeropuertos']
colecc = mydb2['data']

y = colecc.insert_many(cur)

In [36]:
print(y.inserted_ids)

[{'aeropuerto': 'MDW', 'periodo': '2016/1'}, {'aeropuerto': 'MSP', 'periodo': '2016/1'}, {'aeropuerto': 'BWI', 'periodo': '2016/1'}, {'aeropuerto': 'PDX', 'periodo': '2016/1'}, {'aeropuerto': 'IAD', 'periodo': '2016/1'}, {'aeropuerto': 'SAN', 'periodo': '2016/1'}, {'aeropuerto': 'CLT', 'periodo': '2016/1'}, {'aeropuerto': 'LGA', 'periodo': '2016/1'}, {'aeropuerto': 'TPA', 'periodo': '2016/1'}, {'aeropuerto': 'FLL', 'periodo': '2016/1'}, {'aeropuerto': 'BOS', 'periodo': '2016/1'}, {'aeropuerto': 'LAS', 'periodo': '2016/1'}, {'aeropuerto': 'SEA', 'periodo': '2016/1'}, {'aeropuerto': 'JFK', 'periodo': '2016/1'}, {'aeropuerto': 'MDW', 'periodo': '2015/12'}, {'aeropuerto': 'EWR', 'periodo': '2015/12'}, {'aeropuerto': 'BWI', 'periodo': '2015/12'}, {'aeropuerto': 'TPA', 'periodo': '2015/12'}, {'aeropuerto': 'SAN', 'periodo': '2015/12'}, {'aeropuerto': 'SLC', 'periodo': '2015/12'}, {'aeropuerto': 'DEN', 'periodo': '2015/12'}, {'aeropuerto': 'DTW', 'periodo': '2015/12'}, {'aeropuerto': 'DCA', '